In [ ]:
# data formatting scripts
from collections import defaultdict
import json, math, glob
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
from tqdm import tqdm


In [ ]:
import sys
from datetime import datetime
import os

# sys.path.append('raw-data/lcb_submissions')
# sys.path.append('raw-data/LiveCodeBench')

# # %cd raw-data/lcb_submissions/

def load_performances_generation(version=6):
    results = []
    # Get current directory to restore later
    original_dir = os.getcwd()
    
    os.chdir('raw-data/lcb_submissions')

    for model in glob.glob("*/Scenario.codegeneration_*_eval_all.json"):
        mname = model.split("/")[0]
        print(mname)
        fnames = [
            f"{mname}/Scenario.codegeneration_*_eval_all.json"
            # for i in range(9)
        ]
        fname = sum([glob.glob(fname) for fname in fnames], [])

        if not fname:
            print(fnames, "not found")
            # print(f"{fname} does not exist")
            fname = f".json"
            fname = glob.glob(fname)
            if not fname:
                # print(f"{fname} does not exist")
                continue
            else:
                assert len(fname) == 1
            fname = fname[0]
        else:
            if len(fname) != 1:
                continue
            fname = fname[0]

        with open(fname) as fp:
            model_outputs = json.load(fp)
        
        lengths = {5: 880, 6:1055}
        if len(model_outputs) != lengths[v]:
            continue
        # assert (
        #     model.release_date is not None
        # ), f"Model {model.model_repr} has no release date"
        results.extend(
            [
                {
                    # "model_class": model,
                    "question_id": model_output["question_id"],
                    "model": mname,
                    "date": datetime.fromisoformat(model_output["contest_date"]),
                    "difficulty": model_output["difficulty"],
                    "count": len(model_output["graded_list"]),
                    "pass@1": (
                        model_output["pass1"] * 100
                        if "pass1" in model_output
                        else model_output["pass@1"] * 100
                    ),
                    "platform": (
                        "leetcode"
                        if isinstance(model_output["question_id"], int)
                        else (
                            "codeforces"
                            if model_output["question_id"][0] == "1"
                            else model_output["platform"]
                        )
                    ),
                }
                for model_output in model_outputs
            ]
        )
    df = pd.DataFrame(results)
    # print(df.head())
    os.chdir(original_dir)
    return df


for v in [5, 6]:
    df = load_performances_generation(version=v)
    display(df)
    dfo = df.copy()
    dfo['benchmark_id'] = f'lcb_codegen_v{v}'
    dfo['example_id'] = dfo['platform'] + '.' + dfo['question_id']
    dfo['pass1'] = dfo['pass@1'] / 100.0
    dfo = dfo[['benchmark_id', 'model', 'example_id', 'pass1', "count", "date"]]
    dfo.to_json(f'data/lcb_codegen_v{v}.jsonl', orient='records', lines=True)
    display(dfo)

In [ ]:
df = load_performances_generation(version=6)
display(df)
df = df[df["date"] > datetime.fromisoformat("2024-08-01")]
display(df.groupby("model").agg("count"))
dfo = df.copy()
dfo['benchmark_id'] = f'lcb_codegen_v{v}_080124'
dfo['example_id'] = dfo['platform'] + '.' + dfo['question_id']
dfo['pass1'] = dfo['pass@1'] / 100.0
dfo = dfo[['benchmark_id', 'model', 'example_id', 'pass1', "count", "date"]]
dfo.to_json(f'data/lcb_codegen_v{v}_080124.jsonl', orient='records', lines=True)
display(dfo)

In [ ]:
import os
import glob
import json
# Get current directory to restore later
original_dir = os.getcwd()

try:
    # Change to the lcb_submissions directory
    os.chdir('raw-data/lcb_submissions')
    # Find all JSON files and count entries
    for json_file in glob.glob('*/*.json'):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    count = len(data)
                elif isinstance(data, dict):
                    count = len(data)
                else:
                    count = 1
                print(f"{json_file}: {count} entries")
        except Exception as e:
            print(f"{json_file}: Error reading file - {e}")
            
finally:
    # Restore original directory
    os.chdir(original_dir)

In [ ]:
def process_swe(swetype, id_path):
    with open(id_path) as id_file:
        res = json.load(id_file)
    print(res)
    ids = set(res['generated']) | set(res['no_generation'])
    print(f'len of {swetype}', len(ids))
    print('model', 'total', 'deduped')
    for fname in glob.glob(f"raw-data/swebench-experiments/evaluation/{swetype}/*/results/results.json"):
        mname = fname.split('/')[-3]
        try:
            with open(id_path) as id_file:
                res = json.load(id_file)
        except:
            print('not jsonl', fname)
        total_list = res["generated"] + res["no_generation"]
        print(mname, len(total_list), len(set(total_list)))
        assert len(total_list) == len(ids), "results for {fname} is incomplete"

    records = []
    for fname in glob.glob(f"raw-data/swebench-experiments/evaluation/{swetype}/*/results/results.json"):
        # print(fname)
        mname = fname.split('/')[-3]
        print(mname)
        with open(fname, 'r') as f:
            result = json.load(f)
        if 'resolved' in result:
            resolved = set(result['resolved'])
        else: resolved = set()
        
        for id in ids:
            records.append({
                'benchmark_id': f'swebench-{swetype}',
                'model': mname,
                'example_id': id,
                'pass1': 1 if id in resolved else 0,
                'count': 1, # all single submission
            })
    dfo = pd.DataFrame(records)
    display(dfo)
    dfo.to_json(f'data/swebench-{swetype}.jsonl', orient='records', lines=True)

swetype = 'lite'
id_path = 'raw-data/swebench-experiments/evaluation/lite/20231010_rag_claude2/results/results.json'
process_swe(swetype, id_path)

swetype = 'verified'
id_path = 'raw-data/swebench-experiments/evaluation/verified/20231010_rag_claude2/results/results.json'
process_swe(swetype, id_path)

swetype = 'test'
id_path = 'raw-data/swebench-experiments/evaluation/test/20231010_rag_claude2/results/results.json'
process_swe(swetype, id_path)

In [ ]:
def get_cruxeval(type, temp="0.2"):
    records = []
    for fname in glob.glob(f"raw-data/crux-eval.github.io/evaluation_results/*_temp{temp}_{type}.json"):
        name = fname.split('/')[-1]
        model, temps, typejsonl = name.split('_')
        print(model, temps, type)

        with open(fname) as f:
            res = json.load(f)['raw_scored_generations']
            for exid in res:
                gotid = np.mean(res[exid])
                K = len(res[exid])
                actualid = exid.replace('sample_', '')
                records.append({
                    'benchmark_id': f'CRUXEval-{type}-T{temp}',
                    'model': model,
                    'example_id': f"CRUXEval-{type}/{actualid}",
                    'pass1': gotid,
                    'count': K,
                    'hyperparams': f'T{temp}'
                })
    df = pd.DataFrame(records)
    display(df.describe())
    return df

def input_output(temp="0.2"):
    with open(f'data/cruxeval_input_T{temp}.jsonl', 'w') as f:
        dfi = get_cruxeval('input', temp=temp)
        f.write(dfi.to_json(orient='records', lines=True))

    with open(f'data/cruxeval_output_T{temp}.jsonl', 'w') as f:
        dfo = get_cruxeval('output', temp=temp)
        f.write(dfo.to_json(orient='records', lines=True))

    pass1 = pd.concat([dfi, dfo])
    display(pass1)

input_output("0.2")
input_output("0.8")

In [ ]:
# generate evalplus links

import sys
from jinja2 import Template

sys.path.append('/private/home/sida/git/CodeGen/scripts/sida/arena/data/evalplus')

from evalplus.data import get_mbpp_plus, get_human_eval_plus, write_jsonl
# has these fields dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'contract', 'base_input', 'atol', 'plus_input'])

j2_template = Template("""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />   <!--It is necessary to use the UTF-8 encoding with plotly graphics to get e.g. negative signs to render correctly -->
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<link
  rel="stylesheet"
  href="https://crux-eval.github.io/static/css/bulma.min.css"
>
</head>

<body>
<section class="section">                       
<h1> {{ id }} </h1>
<pre><code> {{ problem }} </code></pre>
<h2>solution</h2>
<pre><code> {{ canonical_solution }} </code></pre>
<h2>base input</h2>
<pre><code> {{ base_input }} </code></pre>
<h2>plus input</h2>
<pre><code> {{ plus_input }} </code></pre>
<br>
<h1>Generations</h1>
{% for key,value in outputs.iterrows() %}
      <p> {{ value['model'] }}. fail_base: {{ value['fail_base'] }}, fail_plus: {{ value['fail_plus'] }} </p>
      <pre><code>  {{ value['raw_genereation'] }}</code></pre>
{% endfor %}
                       
</section>

</body>
</html>
""")

def write_benchmark(benchmark_id):
    if benchmark_id == 'humaneval+':
        get_bench = get_human_eval_plus
    else:
        get_bench = get_mbpp_plus

    samples = [
        dict(example_id=task_id,
            problem=problem['prompt'],
            canonical_solution=problem['canonical_solution'],
            base_input=problem['base_input'],
            plus_input=problem['plus_input'],
        )
        for task_id, problem in get_bench().items()
    ]
    df_prob = pd.DataFrame(samples)

    records = []
    for fname in glob.glob(f"data/noise_analysis/*/*.jsonl"):
        with open(fname, 'rt') as f:
            records.extend([json.loads(l) for l in f.readlines()])

    eval_results = pd.DataFrame(records)
    display(eval_results.describe())

    benchmark_results = eval_results[eval_results['benchmark_id'] == benchmark_id]
    df = df_prob.merge(benchmark_results, on='example_id')

    ids = list(set(df['example_id']))
    for id in ids:
        current_prob = df[df['example_id'] == id]
        first = current_prob.iloc[0]
        # display(current_prob)
        generation = current_prob[['model', 'raw_genereation', 'sanitized_generation', 'fail_base', 'fail_plus']]

        with open(f'crux-eval.github.io/eval-arena/evalplus/{id}.html', 'w') as f: 
            f.write(j2_template.render({'id': id, 'problem': first['problem'],
                                        'canonical_solution': first['canonical_solution'],
                                        'base_input': first['base_input'],
                                        'plus_input': first['plus_input'],
                                        'outputs': generation}))

write_benchmark('humaneval+')
write_benchmark('mbpp+')

In [ ]:
def inspect(benchmark_id, example_id):
    # has these fields dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'contract', 'base_input', 'atol', 'plus_input'])
    if benchmark_id == 'humaneval+':
        get_bench = get_human_eval_plus
    else:
        get_bench = get_mbpp_plus

    samples = [
        dict(example_id=task_id, problem=problem['prompt'], solution=problem['canonical_solution'], test=problem['test'], plus_input=problem['plus_input'])
        for task_id, problem in get_bench().items()
    ]
    df_prob = pd.DataFrame(samples)
    for r in df_prob[df_prob['example_id'] == example_id].to_numpy():
        for v in r:
            print(v)

inspect('humaneval+', 'HumanEval/122')

In [ ]:
dfi = pd.read_csv('data/lcb_arena.csv')
display(dfi)
with open('data/lcb_arena.jsonl', 'w') as f:
    dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
    f.write(dfi.to_json(orient='records', lines=True, index=False))


In [ ]:
dfi = pd.read_json('raw-data/ds1000-full.jsonl', lines=True)
with open('data/ds1000.jsonl', 'w') as f:
    dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
    f.write(dfi.to_json(orient='records', lines=True))

In [ ]:
# process evalplus ones
def evalplus(name, isplus):
    records = []
    for fname in glob.glob(f"data/noise_analysis/{name}/*.jsonl"):
        with open(fname, 'rt') as f:
            records.extend([json.loads(l) for l in f.readlines()])
    df = pd.DataFrame(records)
    # display(df.describe())
    if isplus:
        df['pass1'] = np.where(df['fail_plus'], 0, 1) 
        df['benchmark_id'] = f'{name}+' 
    else:
        df['pass1'] = np.where(df['fail_base'], 0, 1) 
        df['benchmark_id'] = f'{name}'
    df = df[[
        'benchmark_id',
        'model',
        'example_id',
        'pass1',
    ]]
    return df

with open('data/humaneval.jsonl', 'w') as f:
    dfi = evalplus('humaneval', False)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/humaneval+.jsonl', 'w') as f:
    dfi = evalplus('humaneval', True)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/mbpp.jsonl', 'w') as f:
    dfi = evalplus('mbpp', False)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/mbpp+.jsonl', 'w') as f:
    dfi = evalplus('mbpp', True)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

In [ ]:
# # display(df)
# sel = df[(df["model"] == "llama-3.1-8B-instruct") & (df["benchmark_id"] == "human_eval_plus")]
# display(sel)

In [ ]:
# too few models in these...
EXCLUDES = ["sat_cot", "livecodebench_cot"]

def gather_by_type(type="large-lowk"):
    dfi = pd.read_csv(f"raw-data/0926_evals_share/{type}/results.csv")
    display(dfi.describe())
    display(dfi[["model", "task_name"]].describe())
    df = dfi.rename(columns={"task_name": "benchmark_id", "question_idx": "example_id", 'model': 'model', "count": "count", "correct": "correct", "pass": "pass1"})
    df["example_id"] = df["example_id"].apply(lambda x: str(x))
    ori_set = set(df["benchmark_id"])
    df = df[~df["benchmark_id"].isin(EXCLUDES)]
    display("removed", ori_set - set(df["benchmark_id"]))
    display("keeping", set(df["benchmark_id"]))
    return df

df = gather_by_type("large-lowk")
# df.describe()
with open('data/vllm_evals/lowk_temp0.7.jsonl', 'w') as f:
    f.write(df.to_json(orient='records', lines=True))

df = gather_by_type("small-highk")
# df.describe()
with open('data/vllm_evals/highk_temp0.7.jsonl', 'w') as f:
    f.write(df.to_json(orient='records', lines=True))

df = gather_by_type("temp1")
df.describe()
with open('data/vllm_evals/lowk_temp1.jsonl', 'w') as f:
    f.write(df.to_json(orient='records', lines=True))

In [ ]:
# the data in data/safim.jsonl has duplicate model, benchmark_id, example_id
import utils
df = pd.DataFrame(utils.load_jsonl_files("raw-data/safim.jsonl"))
display(df)
# df["count"] = 1
# reduce by summing the pass1 of each group
df_deduped = df.groupby(['benchmark_id', 'example_id', 'model'], as_index=False).aggregate({"pass1": "max"})
df = df_deduped
with open('data/safim_dedup.jsonl', 'w') as f:
    f.write(df.to_json(orient='records', lines=True))

In [ ]:
# cruxeval has some non-unique counts. will not fix. 
import utils
df = pd.DataFrame(utils.load_jsonl_files("data/cruxeval_input_T0.2.jsonl"))
# display(df)
display(df[df["model"]=="gpt-4o+cot"])
# df["count"] = 1